In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh

In [2]:
mypath = '../../data/MLgSA/wss/'
mypath1 = '../../data/MLgSA/New_wss/'

onlyfiles = np.asarray([f for f in listdir(mypath) if isfile(join(mypath, f))])
onlyfiles1 = np.asarray([f for f in listdir(mypath1) if isfile(join(mypath1, f))])

onlyfiles.sort()
onlyfiles1.sort()

print(len(onlyfiles), len(onlyfiles1))

short_list = np.asarray([s[5:23] if s[19] == 'l' else s[5:24] for s in onlyfiles])
short_list1 = np.asarray([s[5:23] if s[19] == 'l' else s[5:24] for s in onlyfiles1])

109 50


In [4]:
reader = pv.get_reader(mypath1 + onlyfiles1[1])
mesh  = reader.read()

In [5]:
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars = 'longitudinal_WSS_@_t=1.1', show_edges=True, opacity=0.5)
plotter.show()

Widget(value='<iframe src="http://localhost:36763/index.html?ui=P_0x7f75a8eaec10_0&reconnect=auto" class="pyvi…

In [6]:
mesh.point_data['data'] = mesh['longitudinal_WSS_@_t=1.1']

In [7]:
def extract_edges(mesh):
    cells = mesh.n_cells
    cell_types = mesh.celltypes

    edges = set()
    for i  in range(cells):
        if cell_types[i] == 5:  # Assuming triangles
            cell = mesh.get_cell(i)
            edges.add(tuple(sorted(cell.get_edge(0).point_ids)))
            edges.add(tuple(sorted(cell.get_edge(1).point_ids)))
            edges.add(tuple(sorted(cell.get_edge(2).point_ids)))

    edges = np.array(list(edges))
    return edges

In [8]:
edges = extract_edges(mesh)

In [9]:
# Calculate the Laplacian matrix
def compute_laplacian(mesh, edges):
    n_points = mesh.number_of_points

    row_indices = np.hstack([edges[:, 0], edges[:, 1]])
    col_indices = np.hstack([edges[:, 1], edges[:, 0]])
    data = np.ones_like(row_indices)
    
    adjacency_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(n_points, n_points))
    degree_matrix = csr_matrix((np.array(adjacency_matrix.sum(axis=1)).flatten(), (range(n_points), range(n_points))))
    
    laplacian_matrix = degree_matrix - adjacency_matrix
    return laplacian_matrix

laplacian_matrix = compute_laplacian(mesh, edges)

In [10]:
laplacian_matrix = compute_laplacian(mesh, edges)

# Calculate the Laplacian smoothness field (Laplacian applied to the data)
laplacian_field = laplacian_matrix.dot(mesh.point_data['data'])
mesh.point_data['laplacian_field'] = laplacian_field


# Plotting the maximums
max_curv_indices = np.argsort(laplacian_field)[-50:]
max_curv_nodes = mesh.points[max_curv_indices]



# Visualize the Laplacian field
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars='laplacian_field', cmap='coolwarm', show_edges=True, opacity=0.5)
plotter.add_points(max_curv_nodes, color='red', point_size=10)
plotter.show()

Widget(value='<iframe src="http://localhost:36763/index.html?ui=P_0x7f75a4f41ac0_1&reconnect=auto" class="pyvi…

In [11]:
# Calculate and visualize the first few eigenvectors of the Laplacian
k = 3  # Number of eigenvectors to visualize
laplacian_matrix = laplacian_matrix.astype(float)
eigenvalues, eigenvectors = eigsh(laplacian_matrix, k=k, which='SM')
for i in range(k):
    mesh.point_data[f'eigenvector_{i+1}'] = eigenvectors[:, i]


In [12]:
i=2
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars=f'eigenvector_{i+1}', cmap='viridis', show_edges=True, opacity=0.5)
plotter.show()

Widget(value='<iframe src="http://localhost:36763/index.html?ui=P_0x7f75a44208e0_2&reconnect=auto" class="pyvi…

In [13]:
signal = mesh['laplacian_field']

In [14]:
mesh.get_cell(0).point_ids

[0, 1, 2]

In [15]:
# Extract edges from the mesh
edges = mesh.extract_all_edges()

# Compute the gradient (difference) at each edge
edge_gradients = []
for edge in edges.lines.reshape((-1, 3))[:, 1:]:
    point1, point2 = edge
    gradient = np.abs(signal[point1] - signal[point2])
    edge_gradients.append(gradient)

edge_gradients = np.array(edge_gradients)
edges.cell_data['Edge Gradient'] = edge_gradients


In [16]:
# Plot the edges colored by the gradient values
plotter = pv.Plotter()
plotter.add_mesh(edges, scalars='Edge Gradient', cmap='viridis')
plotter.add_scalar_bar(title='Edge Gradient')
plotter.show()


Widget(value='<iframe src="http://localhost:36763/index.html?ui=P_0x7f759a8808b0_3&reconnect=auto" class="pyvi…

In [17]:
from collections import defaultdict

# Initialize a dictionary to store neighbors for each point
neighbors = defaultdict(set)

# Loop over each cell and populate the neighbors dictionary
for i in range(mesh.n_cells):
    cell = mesh.get_cell(i)
    cell_points = cell.point_ids
    for i in range(len(cell_points)):
        for j in range(i + 1, len(cell_points)):
            neighbors[cell_points[i]].add(cell_points[j])
            neighbors[cell_points[j]].add(cell_points[i])


In [18]:
# Visualize Total Variation by highlighting edges with largest differences
# Initialize an array to store the total variation values
total_variation = np.zeros(mesh.number_of_points)

# Compute the total variation for each vertex
for point_id in range(mesh.number_of_points):
    # Get the signal values at the point and its neighbors
    point_signal = signal[point_id]
    neighbor_signals = [signal[neighbor] for neighbor in neighbors[point_id]]
    # Calculate the total variation
    total_variation[point_id] = np.sum(np.abs(point_signal - np.array(neighbor_signals)))

# Assign the total variation values to the mesh
mesh.point_data['total_variation'] = total_variation




max_var_indices = np.argsort(total_variation)[-50:]
max_var_nodes = mesh.points[max_var_indices]

In [19]:
# Plot the mesh colored by the total variation values
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars='total_variation', cmap='viridis', show_edges=True, opacity=0.5)
plotter.add_points(max_var_nodes, color='red', point_size=10)
plotter.show()


Widget(value='<iframe src="http://localhost:36763/index.html?ui=P_0x7f759b4b4760_4&reconnect=auto" class="pyvi…

In [20]:
# Initialize an array to store the Hölder continuity values
holder_continuity = np.zeros(mesh.number_of_points)

# Compute the Hölder continuity for each vertex
for point_id in range(mesh.number_of_points):
    # Get the signal values at the point and its neighbors
    point_signal = signal[point_id]
    point_coord = mesh.points[point_id]
    neighbor_coords = [mesh.points[neighbor] for neighbor in neighbors[point_id]]
    neighbor_signals = [signal[neighbor] for neighbor in neighbors[point_id]]
    
    # Calculate the distances and differences in signal values
    distances = np.linalg.norm(np.array(neighbor_coords) - point_coord, axis=1)
    differences = np.abs(np.array(neighbor_signals) - point_signal)
    
    # Calculate the Hölder continuity exponent alpha
    if len(distances) > 0:
        log_distances = np.log(distances)
        log_differences = np.log(differences)
        alpha = -np.polyfit(log_distances, log_differences, 1)[0]
        holder_continuity[point_id] = alpha
    else:
        holder_continuity[point_id] = 0

# Assign the Hölder continuity values to the mesh
mesh.point_data['holder_continuity'] = holder_continuity



In [21]:
max_hold_indices = np.argsort(holder_continuity)[:50]
max_hold_nodes = mesh.points[max_hold_indices]

In [22]:
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars='holder_continuity', cmap='viridis', show_edges=True, opacity=0.5)
plotter.add_points(max_hold_nodes, color='red', point_size=10)
plotter.show()


Widget(value='<iframe src="http://localhost:36763/index.html?ui=P_0x7f7574dddf70_5&reconnect=auto" class="pyvi…

In [23]:
signal=mesh['longitudinal_WSS_@_t=1.1']

In [24]:
# Initialize an array to store the standard deviation values
std_devs = np.zeros(mesh.number_of_points)

# Compute the standard deviation for each vertex
for point_id in range(mesh.number_of_points):
    # Get the signal values at the point and its neighbors
    values = [signal[point_id]] + [signal[neighbor] for neighbor in neighbors[point_id]]
    # Calculate the standard deviation
    std_devs[point_id] = np.var(values)

# Assign the standard deviation values to the mesh
mesh.point_data['std_dev'] = std_devs


max_std_indices = np.argsort(std_devs)[-50:]
max_std_nodes = mesh.points[max_std_indices]

In [25]:
# Plot the mesh colored by the standard deviation values
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars='std_dev', cmap='viridis', opacity=0.5)
plotter.add_points(max_std_nodes, color='red', point_size=10)
plotter.show()


Widget(value='<iframe src="http://localhost:36763/index.html?ui=P_0x7f756bfca070_6&reconnect=auto" class="pyvi…